In [1]:
import math
import numpy as np
import copy
n = 5

In [2]:
def sort(A,D):
    for i in range(len(A)):
        for j in range (i+1,len(A)):
            if (D[A[i][0]][A[i][1]] > D[A[j][0]][A[j][1]]):
                temp = A[i]
                A[i] = A[j]
                A[j] = temp
    return A

In [3]:
def sort_J(J):
    for i in range(len(J)):
        for j in range (i+1,len(J)):
            if (J[i][2] > J[j][2]):
                temp = J[i]
                J[i] = J[j]
                J[j] = temp
    return J

In [4]:
def Loop_Judge(j):
        
    #動かなくなるまでまとめる。
    flag = True

    while(flag):
        flag =  False
        
        if (len(j) > 0):
            for i in range(len(j)):
                
                if (j[i][0] == j[i][1]):
                    del j[i]
                    flag = True
                
                for k in range(i+1,len(j)):
                
                    if (j[i][0] == j[i][1]):
                        del j[i]
                        flag = True
                
                    elif (j[i][1] == j[k][0]):
                        j.append([j[i][0],j[k][1]])
                        del j[k]
                        del j[i]
                        flag = True
                
                    elif (j[i][0] == j[k][1]):
                        j.append([j[k][0],j[i][1]])
                        del j[k]
                        del j[i]
                        flag = True
                    
                    if (flag): break
                if (flag): break
    
    if (len(j) is 0):
        return True
    else:
        return False

In [5]:
j = [[0,1],[1,2],[2,4],[4,1]]
Loop_Judge(j)

False

In [6]:
Loop_Judge([[2,3],[0,2],[1,0],[3,4],[4,1]])

True

In [7]:
def calc_g(j,data):
    
    D = copy.copy(data)
    end_flag = False
    
    #固定されている部分の処理を行う
    j_ok = j[0]
    j_ng = j[1]
    
    print("j_ok: ",j_ok, "  j_ng: ",j_ng)
    
    #まず、通らない部分の処理
    for i in j_ng:
        D[i[0]][i[1]] = math.inf
    
    #必ず通る部分は、行と列を削除する
    col = []
    row = []
    for i in j_ok:
        col.append(i[0])
        row.append(i[1])
    col = list(set(col))
    row = list(set(row))
    
    D= np.delete(D, col, 0)
    D= np.delete(D, row, 1)
    
    print("表: ")
    print(D)
    
    #Dが2*2の行列のとき、終端させ、gも求める（下端としてではなく、可能解として）
    if (D.shape == (2,2)):
        
        d_col = []
        d_row = []
        for i in range(n):
            if (i not in col):
                d_col.append(i)
            if (i not in row):
                d_row.append(i)
        
        if ((D[0][0] is not math.inf) and (D[1][1] is not math.inf)):
            if Loop_Judge(j_ok + [[d_col[0],d_row[0]],[d_col[1],d_row[1]]]) is True:
            
                g_tmp1 = 0
                for i in j_ok:
                    g_tmp1 = g_tmp1 + data[i[0]][i[1]]
                g_tmp1 = g_tmp1 + D[0][0] + D[1][1]
                
                end_flag1 = True
                
        if ((D[0][1] is not math.inf) and (D[1][0] is not math.inf)):
            if Loop_Judge(j_ok + [[d_col[0],d_row[1]],[d_col[1],d_row[0]]]) is True:

                g_tmp2 = 0
                for i in j_ok:
                    g_tmp2 = g_tmp2 + data[i[0]][i[1]]
                g_tmp2 = g_tmp2 + D[1][0] + D[0][1]
                
                end_flag2 = True
                
        if end_flag1 == True and end_flag2 == True:
            g = min(g_tmp1,g_tmp2)
        elif end_flag1 == True and end_flag2 == False:
            g = g_tmp1
        elif end_flag1 == False and end_flag2 == True:
            g = g_tmp2
        elif end_flag1 == False and end_flag2 == False:
            g = math.inf
            
        end_flag = True
    
    #その他の場合、gの計算を行う
    else:
        col_min = D.min(axis=1)
        for i in range(len(col_min)):
            D[i] = D[i] - col_min[i]
    
        row_min = D.min(axis=0)
    
        g = col_min.sum()+row_min.sum()
        for i in j_ok:
            g = g + data[i[0]][i[1]]
    
    return [g,end_flag]

In [29]:
data = np.array([[math.inf,21,7,13,15],
              [11,math.inf,19,12,25],
              [15,24,math.inf,13,5],
              [6,17,9,math.inf,22],
              [28,6,11,5,math.inf]])
calc_g([[[2,4],[3,0]],[[4,3]]],data)

j_ok:  [[2, 4], [3, 0]]   j_ng:  [[4, 3]]
表: 
[[ 21.   7.  13.]
 [ inf  19.  12.]
 [  6.  11.  inf]]


[36.0, False]

In [30]:
calc_g([[[2,4],[4,1],[3,0]],[[4,3]]],data)

j_ok:  [[2, 4], [4, 1], [3, 0]]   j_ng:  [[4, 3]]
表: 
[[  7.  13.]
 [ 19.  12.]]


[36.0, True]

In [31]:
k = calc_g([[],[]],data)
k

j_ok:  []   j_ng:  []
表: 
[[ inf  21.   7.  13.  15.]
 [ 11.  inf  19.  12.  25.]
 [ 15.  24.  inf  13.   5.]
 [  6.  17.   9.  inf  22.]
 [ 28.   6.  11.   5.  inf]]


[35.0, False]

In [28]:
#最適な時の固定経路から、移動経路を出力する

def GetPath(j,data):
    
    D = copy.copy(data)
    root = 0
    j_ok = j[0]
    j_ng = j[1]
    
    #まず、通らない部分の処理
    for i in j_ng:
        D[i[0]][i[1]] = math.inf
    
    #必ず通る部分は、行と列を削除する
    col = []
    row = []
    for i in j_ok:
        col.append(i[0])
        row.append(i[1])
    col = list(set(col))
    row = list(set(row))
    
    D= np.delete(D, col, 0)
    D= np.delete(D, row, 1)
        
    d_col = []
    d_row = []
    for i in range(n):
        if (i not in col):
            d_col.append(i)
        if (i not in row):
            d_row.append(i)
        
    if ((D[0][0] is not math.inf) and (D[1][1] is not math.inf)):
        if Loop_Judge(j_ok + [[d_col[0],d_row[0]],[d_col[1],d_row[1]]]) is True:
            
            g_tmp1 = 0
            for i in j_ok:
                g_tmp1 = g_tmp1 + data[i[0]][i[1]]
            g_tmp1 = g_tmp1 + D[0][0] + D[1][1]
                
            end_flag1 = True
                
    if ((D[0][1] is not math.inf) and (D[1][0] is not math.inf)):
        if Loop_Judge(j_ok + [[d_col[0],d_row[1]],[d_col[1],d_row[0]]]) is True:

            g_tmp2 = 0
            for i in j_ok:
                g_tmp2 = g_tmp2 + data[i[0]][i[1]]
            g_tmp2 = g_tmp2 + D[1][0] + D[0][1]
                
            end_flag2 = True
                
    if end_flag1 == True and end_flag2 == True:
        if g_tmp1 < g_tmp2:
            root = 1
        else:
            root = 2
                
    elif end_flag1 == True and end_flag2 == False:
        root = 1
    elif end_flag1 == False and end_flag2 == True:
        root = 2
    
    if root == 1:
        return j_ok + [[d_col[0],d_row[0]],[d_col[1],d_row[1]]]
    elif root == 2:
        return j_ok + [[d_col[0],d_row[1]],[d_col[1],d_row[0]]]


In [32]:
def TSP(A,J,data):
    temp_result = math.inf
    temp_j = None
    
    while (len(J) is not 0):
        
        J = sort_J(J)        
        j = J.pop(0)
        if (j[2] > temp_result):
            continue
        
        result = calc_g(j,data)
        g = float(result[0])
        end_flag = bool(result[1])
        
        print("result:",result)
        
        if end_flag == True:
            if temp_result > g:
                temp_result = g
                temp_j = copy.copy(j)
            continue

        #終端していない時は、Aから一つ取り出して、stockする
        a = A[len(j[0])+len(j[1])]
        j0 = copy.copy(j[0])
        j0.append(a)
        j1 = copy.copy(j[1])
        j1.append(a)
        
        J.append([j0,j[1],g])
        J.append([j[0],j1,g])
    
    print ("---------------------------------------------")
    print ("最短経路の距離：",temp_result)
    print ("移動する経路：",GetPath(temp_j,data))

In [33]:
import math
import numpy as np
n = 5
data = np.array([[math.inf,21,7,13,15],
              [11,math.inf,19,12,25],
              [15,24,math.inf,13,5],
              [6,17,9,math.inf,22],
              [28,6,11,5,math.inf]])

In [34]:
A = [[x,y] for x in range(n) for y in range(n) if x != y]
A = sort(A,data)

In [35]:
a = A.pop(0)
TSP(A,[[[a],[],k[0]],[[],[a],k[0]]],data)

j_ok:  [[2, 4]]   j_ng:  []
表: 
[[ inf  21.   7.  13.]
 [ 11.  inf  19.  12.]
 [  6.  17.   9.  inf]
 [ 28.   6.  11.   5.]]
result: [35.0, False]
j_ok:  []   j_ng:  [[2, 4]]
表: 
[[ inf  21.   7.  13.  15.]
 [ 11.  inf  19.  12.  25.]
 [ 15.  24.  inf  13.  inf]
 [  6.  17.   9.  inf  22.]
 [ 28.   6.  11.   5.  inf]]
result: [51.0, False]
j_ok:  [[2, 4], [4, 1]]   j_ng:  []
表: 
[[ inf   7.  13.]
 [ 11.  19.  12.]
 [  6.   9.  inf]]
result: [36.0, False]
j_ok:  [[2, 4]]   j_ng:  [[4, 1]]
表: 
[[ inf  21.   7.  13.]
 [ 11.  inf  19.  12.]
 [  6.  17.   9.  inf]
 [ 28.  inf  11.   5.]]
result: [45.0, False]
j_ok:  [[2, 4], [4, 1], [3, 0]]   j_ng:  []
表: 
[[  7.  13.]
 [ 19.  12.]]
result: [36.0, True]
j_ok:  [[2, 4], [4, 1]]   j_ng:  [[3, 0]]
表: 
[[ inf   7.  13.]
 [ 11.  19.  12.]
 [ inf   9.  inf]]
result: [39.0, False]
---------------------------------------------
最短経路の距離： 36.0
移動する経路： [[2, 4], [4, 1], [3, 0], [0, 2], [1, 3]]
